In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())

【訊息】目前的工作目錄: D:\Mypython\專題


### 範例

In [2]:
a =[1,2,3,4,1,2]
c = ['This','is','a','random','string','!']
df = pd.DataFrame({'A':a,'C':c})
df

,A,C
0,1,This
1,2,is
2,3,a
3,4,random
4,1,string
5,2,!


In [3]:
QQ = df.groupby('A')['C'].apply(lambda x: "%s" % ''.join(x))
QQ

A
1    Thisstring
2           is!
3             a
4        random
Name: C, dtype: object

In [4]:
QQ = list(QQ)
QQ

['Thisstring', 'is!', 'a', 'random']

### 實作-資料清理與分群

In [5]:
#開啟欄位到最大，方便 pandas 匯入檔案
csv.field_size_limit(int(sys.maxsize/10000000000))

131072

In [9]:
#檔案的匯入
df=pd.read_csv(r'D:\Mypython\專題\dealQQ.csv', sep=',', engine='python')

In [10]:
df.shape

(546688, 44)

In [17]:
df.columns

Index(['business_hour_guid', 'seller_GUID', 'business_hour_order_time_s',
       'business_hour_order_time_e', 'business_hour_deliver_time_s',
       'business_hour_deliver_time_e', 'pic_alt', 'zip_code', 'city_name',
       'district_name', 'item_name', 'item_orig_price', 'item_price',
       'item_default_daily_amount', 'event_name', 'event_title',
       'product_spec', 'shopping_cart', 'combo_pack_count',
       'quantity_multiplier', 'city_list', 'deal_acc_business_group_id',
       'acc_business_group_name', 'deal_type', 'is_travel_deal',
       'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'shippingdate', 'is_long_contract',
       'develope_sales_id', 'operation_sales_id', 'slug', 'ordered_quantity',
       'ordered_total', 'category_list', 'freight_amount', 'enable_isp',
       'discount_price', 'is_wms', 'ancestor_business_hour_guid', 'code_name',
       'code'],
      dtype='object')

In [11]:
df.head(2)

,business_hour_guid,seller_GUID,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,pic_alt,zip_code,city_name,district_name,...,ordered_quantity,ordered_total,category_list,freight_amount,enable_isp,discount_price,is_wms,ancestor_business_hour_guid,code_name,code
0,29B37E53-021C-48E9-909E-C6C151974AC7,EC5DDAD4-F8F8-418B-8AED-7BAC2553F2B6,00:00.0,00:00.0,00:00.0,00:59.0,柯絲妃兒/胎盤素/肌因賦活/身體精華乳,407.0,台中市,西屯區,...,0.0,0.0,[88|221|124|769|136|139],0.0,0.0,NaN,0.0,NaN,美體私密,5821.0
1,2C6E83A9-9785-4593-A759-07A78AB309BB,B6CC8A1E-37C8-44E5-B6E2-15123387AF95,00:00.0,00:00.0,00:00.0,00:59.0,aibo 領導者/ 立體聲/智慧藍牙/耳機麥克風系列,243.0,新北市,泰山區,...,0.0,0.0,[88|221|117|2179|2180|100105|100106|136|139],0.0,0.0,NaN,0.0,NaN,網路設備/穿戴裝置/辦公設備,222.0


In [13]:
# 匯入停用詞庫，放進list
stopword =[]
for i in open(r'D:\Mypython\專題\停用.txt') :
    stopword.append(i.rstrip())

In [35]:
# 轉化欄為型態
df['item_name'] = df['item_name'].astype('str')

In [75]:
df['code_name'] = df['code_name'].astype('str').apply(lambda x:x.replace('/',''))

In [76]:
# 將資料去除停用詞
def clearword(x):
    l = []
    out =''
    l = jieba.cut(x)
    for j in l:
        if j not in stopword:
            out = out+j
    return out

In [77]:
# 清理
df['item_name'] = df['item_name'].apply(lambda x:x.replace(' ','').replace('(','').replace(')','').replace('|',''))

In [78]:
# 停用詞去除
df['item_name'] = df['item_name'].apply(clearword)

In [79]:
# 2個欄位結合
df['name'] = df['item_name']+df['code_name']

In [80]:
df['name']

0                       柯絲妃兒胎盤素肌因賦活身體精華乳美體私密
1                領導者立體聲智慧藍牙耳機麥克風網路設備穿戴裝置辦公設備
2                領導者立體聲智慧藍牙耳機麥克風網路設備穿戴裝置辦公設備
3                領導者立體聲智慧藍牙耳機麥克風網路設備穿戴裝置辦公設備
4                領導者立體聲智慧藍牙耳機麥克風網路設備穿戴裝置辦公設備
5                領導者立體聲智慧藍牙耳機麥克風網路設備穿戴裝置辦公設備
6                        黑橋牌招牌香腸原味蒜味即食調理熟食小吃
7                        黑橋牌招牌香腸原味蒜味即食調理熟食小吃
8                        黑橋牌招牌香腸原味蒜味即食調理熟食小吃
9                        黑橋牌招牌香腸原味蒜味即食調理熟食小吃
10                       黑橋牌招牌香腸原味蒜味即食調理熟食小吃
11                             水林號冰心地瓜農產蔬菜水果
12                 膳魔師超輕量鏽鋼真空保溫杯保溫生活百貨衛浴設備用品
13                                 匯鮮鮭魚生鮮肉品蛋
14                   拳擊有氧健身特厚多功能拳擊沙袋健身器材測量器材
15                   拳擊有氧健身直立式拳擊球拳擊座健身器材測量器材
16                   拳擊有氧健身特厚多功能拳擊沙袋健身器材測量器材
17             拳擊有氧健身特厚多功能拳擊沙袋半指拳擊手套健身器材測量器材
18                     尚順育樂一票暢遊尚順晴天雨天玩HIGH門票
19                                尚順育樂門票四張門票
20             拳擊有氧健身直立式拳擊球拳擊座競技拳擊手套健身器材測量器材
21        iPhone7iPhone7plus0.28超薄軟殼電腦平板手機配件
22        

In [81]:
# 同類別的groupby在一起
Join_name = df.groupby('code')['item_name'].apply(lambda x: "%s" % ''.join(x))

In [82]:
Join_name = list(Join_name)

In [83]:
Join_name[0]

"芝麻街夏日鋁箔壓花涼墊L號芝麻街夏日鋁箔壓花涼墊L號芝麻街夏日鋁箔壓花涼墊M號芝麻街夏日鋁箔壓花涼墊L號芝麻街夏日鋁箔壓花涼墊芝麻街夏日鋁箔壓花涼墊M號芝麻街夏日鋁箔壓花涼墊M號寵物項圈墊寵物汽車防汙墊寵物汽車防汙墊寵物汽車防汙墊寵物汽車防汙墊寵物汽車防汙墊昊伸涼席寵物窩小搖籃昊伸編織竹席雙面冰墊3L昊伸編織竹席雙面冰墊L昊伸寵物涼蓆昊伸折疊式寵物涼墊L昊伸編織竹席雙面冰墊昊伸折疊式寵物涼墊菱牌寵物剪毛菱牌充插寵物電動剪毛薄刀菱牌充插寵物電動剪毛大厚刀菱牌插電型寵物剪毛薄刀凝膠寵物墊鋁箔壓花冰墊鋁箔寵物墊中凝膠寵物墊特大凝膠寵物墊凝膠寵物墊特大夏日寵物墊鋁箔寵物墊中鋁箔壓花冰墊寵物吸盤吊床睡床寵物吸盤吊床睡床寵物吸盤吊床睡床寵物吸盤吊床睡床鋁箔壓花冰墊鋁箔壓花冰墊FURminator神奇除毛梳犬用L號FURminator科德士寵物毛梳電剪科德士寵物電推剪FURminator神奇除毛梳犬用XS號FURminator神奇除毛梳貓M號防水靜音寵物專業電剪FURminator神奇除毛梳犬用號FURminator神奇除毛梳貓S號凝膠寵物墊夏日寵物墊鋁箔寵物墊中凝膠寵物墊凝膠寵物墊特大鋁箔寵物墊中凝膠寵物墊特大ChilliPet羊羔絨舒適圓窩入ChilliPet羊羔絨舒適圓窩入ChilliPet羊羔絨舒適圓窩ChilliPet羊羔絨舒適圓窩入美樂狗寵物套餐美樂狗寵物套餐美樂狗寵物套餐美樂狗寵物套餐柔軟南瓜寵物抽繩窩柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入柔軟南瓜寵物抽繩床窩入寵物項圈墊時尚項圈桃紅水鑽項圈寵物小綿羊造型床墊寵物貪睡狗造型床墊寵物貪睡狗造型床墊時尚項圈桃紅水鑽項圈寵物花朵造型床墊寵物花朵造型床墊寵物小綿羊造型床墊愛羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛中號羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛中號羊羔絨五彩寵物窩愛羊羔絨五彩寵物窩愛中號羊羔絨五彩寵物窩愛蒙古包寵物窩愛中號蒙古包寵物窩愛蒙古包寵物窩愛中號蒙古包寵物窩愛蒙古包寵物窩愛中號蒙古包寵物窩愛蒙古包寵物窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩羊羔絨寵物保暖窩柔軟抽繩寵物墊窩柔軟抽繩寵物墊窩柔軟抽繩寵物墊窩柔軟抽

In [85]:
pd.DataFrame(Join_name).to_csv(r"D:\MyPython\專題\Join-name.csv", encoding='utf_8_sig')

### Tf-idf 抽取

In [91]:
df=pd.read_csv(r'D:\Mypython\專題\Join-name.csv', sep=',', engine='python')
df = df.iloc[:,1:]

In [96]:
df.columns

Index(['join_name'], dtype='object')

In [97]:
# 結巴切段
def word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [100]:
# 資料型態轉換
df['join_name'] = df['join_name'].astype('str')

In [101]:
df['content_cutted'] = df['join_name'].apply(word_cut)

In [105]:
# 建立 tf-idf 模型
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.85,
                                min_df = 10)

In [106]:
# 資料餵進模型中
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [107]:
# 觀看模型的產出
tf_feature_names = tf_vectorizer.get_feature_names()
print(len(tf_feature_names))
tf_feature_names

740


['10',
 '100ml',
 '108',
 '10ml',
 '128',
 '140',
 '160',
 '192',
 '216',
 '280',
 '288',
 '2kg',
 '2xl',
 '320',
 '33',
 '350g',
 '399',
 '3c',
 '3d',
 '480',
 '499',
 '72',
 '800',
 '8g',
 '99',
 'av',
 'cd',
 'cuisinart',
 'disney',
 'diy',
 'fujigrace',
 'hellokitty',
 'justbuy',
 'led',
 'mini',
 'mp3',
 'nan',
 'plus',
 'recolte',
 'sformula',
 'spa',
 'usb',
 'uv',
 'wifi',
 'x10',
 'x2',
 'x4',
 'x6',
 '一件',
 '一入',
 '一共',
 '一口',
 '一日',
 '一箱',
 '一送',
 '七分',
 '三合一',
 '三天',
 '三層',
 '三明治',
 '三盒',
 '三種',
 '三角',
 '三送',
 '上市',
 '上衣',
 '不銹鋼',
 '乳酸菌',
 '二人',
 '二件',
 '二合一',
 '二盒',
 '五入',
 '五合一',
 '五送',
 '人份',
 '人氣',
 '人體',
 '件套',
 '任選共',
 '伊萊',
 '休閒',
 '伸縮',
 '低溫',
 '促銷',
 '保健',
 '保暖',
 '保溫',
 '保溫杯',
 '保潔',
 '保濕',
 '保護',
 '保護套',
 '保養',
 '保鮮',
 '保鮮盒',
 '修護',
 '修身',
 '健康',
 '健身',
 '偵測',
 '傳輸線',
 '優惠組',
 '充氣',
 '充電',
 '充電器',
 '充電式',
 '先生',
 '光觸',
 '克斯',
 '克特',
 '兒童',
 '入組',
 '入裝',
 '入超',
 '內衣',
 '全功能',
 '兩件',
 '兩入',
 '兩色',
 '公升',
 '冰晶',
 '冰淇淋',
 '冰箱',
 '冰絲',
 '冷凝',
 '冷氣',
 '凝膠',
 '出清',
 '出貨

In [112]:
outf=open(file=r'D:\MyPython\專題\分群抽取.txt',mode="wt")

In [113]:
for i in tf_feature_names:
    outf.write(str(i)+"\n")
outf.close()